```
    Домашнее задание №2 по курсу Математические основы машинного обучения.
    Подготовил Касаткин Н.Е. Email: kasatkin.ne@phystech.edu
```

Автору этой работы очень хотелось написать нейросеть с использованием популярного фреймворка *Pytorch Lightning*.
Пускай решение и не набрало много очков на kaggle (не хватило времени развить идею), но был получен полезный практический опыт написания небольшого проекта с использованием современных решений.

### Архитектура

В директории `./NeuralNetwork` содержатся скрипты для загрузки датасета, предобработки, деления его на обучение и валидацию; сама нейросеть и необходимые для pytorch lightning модули (`module.py`, `datamodule.py`), а также `loss.py` и `train.py`.

Архитектура нейросети:
```
    nn.Linear(input_size, input_size),
    nn.PReLU(),
    nn.BatchNorm1d(input_size),
    nn.Dropout(0.5),
    nn.Linear(input_size, input_size),
    nn.PReLU(),
    nn.BatchNorm1d(input_size)
    nn.Linear(input_size, num_classes)
```
где `input_size = 100`, `num_classes = 30`

В качестве лосс функции для этой задачи многоклассовой классификации была выбрана стандартная `nn.CrossEntropyLoss()`.

### Предобработка

Был проведен отбор обьектов с использованием простой эвристики - раз данные получены из распределений Дирихле с разными параметрами и быть может шумом, то выкинем все обьекты, которые имеют хотя бы один отрицательный признак и все, которые имеют отрицательную сумму признаков. Таким образом из 10000 обьектов осталась 9000 обьектов, которые реально относятся к распределению Дирихле (это дало существенный прирост качества предсказаний нейросети). В тестовых данных таких обьектов не было.

Из `./NeuralNetwork/dataset.py`:

```
    df = data[features].apply(lambda x: np.sum(x), axis=1)
    data = data[df > 0]
    data = data[np.all(data >= 0, axis=1)]
```

### Обучение

При обучении модели данные были поделены на обучение и валидацию (без контроля) в соотношении 75/25.  
Наилучшая эпоха обучения выбиралась по наименьшему лоссу на валидации (см. `train.py`, `es_callback`).  
Для обучения написан shell скрипт `run_train.sh`.

Точность на валидации составила `68%`, а на тестовой части (на kaggle) `56%`. Это говорит о переобучении модели под валидационные данные, однако этого результата достаточно для оценки 3 из 5 и в силу нехватки времени решил оставить так.

### Получение предсказаний

In [1]:
import os
import os.path as osp
import sys

import numpy as np
import pandas as pd
import torch

from NeuralNetwork.network import NNClassifier
from NeuralNetwork.dataset import MyDataset


# Было проведено несколько экспериментов и выбран наилучший
EXPERIMENT = 'clean_objects'
CHPT = 549

# Название для сохранения весов
SAVE_NAME = 'nnclassifier'

# Загрузка чекпоинта и сохранения файла с весами
ckpt_file = f'output/checkpoints/{EXPERIMENT}/epoch={CHPT}.ckpt'
weights_file = f'models/{SAVE_NAME}.pth'

ckpt = torch.load(ckpt_file, map_location='cpu')
state_dict = {k[9:] if k.startswith('backbone.') else k: v for k, v in ckpt['state_dict'].items()}
torch.save({'model': state_dict}, weights_file)

# Загрузка тестовой части
X, features = MyDataset.load_data('data/test.csv', test=True)
X = torch.from_numpy(X).float()

# Инициализация модели и загрузка весов
model = NNClassifier()
ckpt = torch.load(weights_file, map_location='cpu')
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [2]:
# Получение предсказания из логитов
def predict(logits):
    return torch.argmax(logits, dim=1).numpy()

preds = predict(model(X))

# Запись в файл
with open('output/answer.csv', 'w') as f:
    f.write('Id,Category\n')
    for i, pred in enumerate(preds):
        string = f"{i},{int(pred)}\n"
        f.write(string)

### Заключение

Несмотря на не самый высокий результат на kaggle, хочется заметить, что даже такая простая нейросеть с минимальной предобработкой данных значительно лучше решает подобные задачи "в лоб", чем классические метрические методы такие как kNN (без всяких сложных трюков).  

Я доволен проведенной работой, поскольку очень хотел написать сетку на pytorch lightning и это у меня получилось. 

### Дополнительно

Я, разумеется, пробовал и другие подходы к решению задачи, чтобы просто сравнить их.  
Ниже приведен код для классификации с помощью kNN.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

# Загрузка данных
data = pd.read_csv('data/train.csv', index_col='Id')
features = data.columns[:-1].values.tolist()

# Предобработка (см. начало ноутбука)
df = data[features].apply(lambda x: np.sum(x), axis=1)
data = data[df > 0]
data = data[np.all(data >= 0, axis=1)]

X, y = data[features].values, data['Category'].values
print(f'Total samples: {len(data)}')
counts = np.unique(data["Category"].values, return_counts=True)[1]
print(f'Samples per class\n\tmean: {np.mean(counts):.0f}\n\tmedian: {np.median(counts):.0f}')

data.head()

Total samples: 9000
Samples per class
	mean: 300
	median: 304


,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_92,x_93,x_94,x_95,x_96,x_97,x_98,x_99,x_100,Category
Id,,,,,,,,,,,,,,,,,,,,,
1,9.124391e-31,8.036040e-38,4.614826e-56,2.279895e-99,1.610577e-47,8.487204e-10,2.250473e-11,5.756725e-29,4.040463e-26,2.525177e-85,...,3.152788e-25,7.743067e-78,2.817278e-24,0.000000e+00,0.000000e+00,3.171895e-30,0.000000e+00,6.625910e-22,0.000000e+00,22
2,2.973061e-111,6.405377e-95,2.735872e-11,4.966691e-27,4.218541e-110,2.161913e-107,4.711781e-283,2.057877e-73,2.553696e-12,1.392694e-85,...,1.265311e-08,1.686664e-28,3.263632e-202,8.574362e-20,1.494268e-161,9.315573e-19,0.000000e+00,5.455236e-11,0.000000e+00,25
3,2.168353e-181,5.292826e-65,9.444697e-318,1.390934e-141,1.902337e-23,0.000000e+00,7.248176e-05,1.120929e-210,1.887261e-39,2.409483e-69,...,4.203073e-51,2.046047e-210,0.000000e+00,1.628975e+00,0.000000e+00,1.189557e-80,6.318885e-02,9.980264e-251,1.076291e-34,9
4,2.203527e-128,5.605085e-61,5.309238e-66,3.106777e-82,2.169759e-115,1.535179e-57,1.789653e-250,7.359979e-15,2.533689e-27,2.325256e-205,...,5.316639e-09,1.094104e-82,2.314366e-05,4.846996e-77,5.473915e-241,4.294852e-88,0.000000e+00,1.580658e-15,0.000000e+00,23
5,2.702053e-24,3.426321e-79,1.491430e-18,9.167029e-02,4.237540e-03,5.383225e-48,0.000000e+00,1.552436e-31,0.000000e+00,2.900905e+00,...,1.875603e-216,5.020648e-05,1.608720e-68,1.518459e-17,2.026061e-162,7.991754e-86,1.468795e-12,1.433941e-10,2.561009e-37,16


Деление на обучение, валидацию и контроль.

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val,
                                                  test_size=0.25, shuffle=True)

In [5]:
clf = KNeighborsClassifier(metric='chebyshev')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
print(f'Val  | {(y_pred == y_val).mean()}')

y_pred = clf.predict(X_test)
print(f'Test | {(y_pred == y_test).mean()}')

Val  | 0.43833333333333335
Test | 0.4461111111111111


На kaggle такое решение получает точность `43%`.